In [8]:
# ==========================================================
# Importando as bibliotecas necessárias
# ==========================================================
import requests
import pandas as pd
import requests
import pandas as pd
import time
from urllib.parse import quote_plus
from requests.exceptions import RequestException

In [9]:
# Lendo a tabela para um DataFrame aux
df_cidades_paises = pd.read_csv("df_fato_pais.csv")  
# Agora os dois DataFrames estão disponíveis no mesmo contexto
display(df_cidades_paises)

,nome_resumido,nome,nome_oficial,capital,regiao,subregiao,populacao,area_km2,idiomas,moedas,bandeira_url,data_carga
0,RU,Russia,Russian Federation,Moscow,Europe,Eastern Europe,144104080,17098242.00,Russian,Russian ruble,https://flagcdn.com/w320/ru.png,2025-05-03 13:01:00
1,AQ,Antarctica,Antarctica,Não informado,Antarctic,Não informado,1000,14000000.00,Não informado,Não informado,https://flagcdn.com/w320/aq.png,2025-05-03 13:01:00
2,CA,Canada,Canada,Ottawa,Americas,North America,38005238,9984670.00,"English, French",Canadian dollar,https://flagcdn.com/w320/ca.png,2025-05-03 13:01:00
3,CN,China,People's Republic of China,Beijing,Asia,Eastern Asia,1402112000,9706961.00,Chinese,Chinese yuan,https://flagcdn.com/w320/cn.png,2025-05-03 13:01:00
4,US,United States,United States of America,"Washington, D.C.",Americas,North America,329484123,9372610.00,English,United States dollar,https://flagcdn.com/w320/us.png,2025-05-03 13:01:00
...,...,...,...,...,...,...,...,...,...,...,...,...
245,CC,Cocos (Keeling) Islands,Territory of the Cocos (Keeling) Islands,West Island,Oceania,Australia and New Zealand,544,14.00,English,Australian dollar,https://flagcdn.com/w320/cc.png,2025-05-03 13:01:00
246,TK,Tokelau,Tokelau,Fakaofo,Oceania,Polynesia,1411,12.00,"English, Samoan, Tokelauan",New Zealand dollar,https://flagcdn.com/w320/tk.png,2025-05-03 13:01:00
247,GI,Gibraltar,Gibraltar,Gibraltar,Europe,Southern Europe,33691,6.00,English,Gibraltar pound,https://flagcdn.com/w320/gi.png,2025-05-03 13:01:00
248,MC,Monaco,Principality of Monaco,Monaco,Europe,Western Europe,39244,2.02,French,Euro,https://flagcdn.com/w320/mc.png,2025-05-03 13:01:00


In [10]:
# Convertendo para lista de tuplas
cidades_paises = [tuple(row) for row in df_cidades_paises]

# Exibindo o resultado
display(cidades_paises)

[('n', 'o', 'm', 'e', '_', 'r', 'e', 's', 'u', 'm', 'i', 'd', 'o'),
 ('n', 'o', 'm', 'e'),
 ('n', 'o', 'm', 'e', '_', 'o', 'f', 'i', 'c', 'i', 'a', 'l'),
 ('c', 'a', 'p', 'i', 't', 'a', 'l'),
 ('r', 'e', 'g', 'i', 'a', 'o'),
 ('s', 'u', 'b', 'r', 'e', 'g', 'i', 'a', 'o'),
 ('p', 'o', 'p', 'u', 'l', 'a', 'c', 'a', 'o'),
 ('a', 'r', 'e', 'a', '_', 'k', 'm', '2'),
 ('i', 'd', 'i', 'o', 'm', 'a', 's'),
 ('m', 'o', 'e', 'd', 'a', 's'),
 ('b', 'a', 'n', 'd', 'e', 'i', 'r', 'a', '_', 'u', 'r', 'l'),
 ('d', 'a', 't', 'a', '_', 'c', 'a', 'r', 'g', 'a')]

In [11]:
# ==========================================
# chave da API OpenWeatherMap
# ==========================================
api_key = "e0b696d684ea5b8a15f7f141ca979bc3"  
# ==========================================
# Lista de cidades + código do país
# ==========================================
cidades_paises = [
    (row['capital'], row['nome_resumido'])
    for _, row in df_cidades_paises.iterrows()
    if pd.notna(row['capital']) and pd.notna(row['nome_resumido'])
]

# Remove duplicatas, se houver
cidades_paises = list(set(cidades_paises))

# ==========================================
# Lista para armazenar os dados do clima
# ==========================================
dados_clima = []

# ==========================================
# Loop para consultar a API OpenWeatherMap
# ==========================================
for cidade, codigo_pais in cidades_paises:
    cidade_codificada = quote_plus(cidade)  # trata acentos e espaços

    for tentativa in range(3):  # tenta até 3 vezes por cidade
        try:
            # Monta a URL e os parâmetros
            url = "https://api.openweathermap.org/data/2.5/weather"
            params = {
                "q": f"{cidade_codificada},{codigo_pais}",
                "appid": api_key,
                "units": "metric",
                "lang": "pt_br"
            }

            # Chamada à API
            resposta = requests.get(url, params=params, timeout=10)
            resposta.raise_for_status()  # lança erro se não for 200

            # Converte resposta para JSON
            clima_info = resposta.json()

            # Adiciona dados ao resultado final
            dados_clima.append({
                "cidade": cidade,
                "codigo_pais": clima_info['sys']['country'],
                "temperatura": clima_info['main']['temp'],
                "sensacao_termica": clima_info['main']['feels_like'],
                "temperatura_min": clima_info['main']['temp_min'],
                "temperatura_max": clima_info['main']['temp_max'],
                "umidade": clima_info['main']['humidity'],
                "descricao_clima": clima_info['weather'][0]['description'],
                "horario_medicao": clima_info['dt']  # corrigido: 'dt' fica na raiz
            })

            break  # sucesso, sai do loop de tentativas

        except RequestException as e:
            print(f"[{cidade}, {codigo_pais}] Tentativa {tentativa + 1} falhou: {e}")
            time.sleep(3)  # espera antes de tentar novamente

        except Exception as e:
            print(f"[{cidade}, {codigo_pais}] Erro inesperado: {e}")
            break  # sai do loop se for erro inesperado

    time.sleep(1)  # evita sobrecarga na API (1,5s entre chamadas)

# ==========================================
# Converte resultados em DataFrame
# ==========================================
df_clima = pd.DataFrame(dados_clima)
print(df_clima)



[Sana'a, YE] Tentativa 1 falhou: 404 Client Error: Not Found for url: https://api.openweathermap.org/data/2.5/weather?q=Sana%2527a%2CYE&appid=e0b696d684ea5b8a15f7f141ca979bc3&units=metric&lang=pt_br
[Sana'a, YE] Tentativa 2 falhou: 404 Client Error: Not Found for url: https://api.openweathermap.org/data/2.5/weather?q=Sana%2527a%2CYE&appid=e0b696d684ea5b8a15f7f141ca979bc3&units=metric&lang=pt_br
[Sana'a, YE] Tentativa 3 falhou: 404 Client Error: Not Found for url: https://api.openweathermap.org/data/2.5/weather?q=Sana%2527a%2CYE&appid=e0b696d684ea5b8a15f7f141ca979bc3&units=metric&lang=pt_br
[Cockburn Town, TC] Tentativa 1 falhou: 404 Client Error: Not Found for url: https://api.openweathermap.org/data/2.5/weather?q=Cockburn%2BTown%2CTC&appid=e0b696d684ea5b8a15f7f141ca979bc3&units=metric&lang=pt_br
[Cockburn Town, TC] Tentativa 2 falhou: 404 Client Error: Not Found for url: https://api.openweathermap.org/data/2.5/weather?q=Cockburn%2BTown%2CTC&appid=e0b696d684ea5b8a15f7f141ca979bc3&units

In [12]:
df_clima['horario_medicao'] = pd.to_datetime(df_clima['horario_medicao'], unit='s').dt.strftime('%d/%m/%Y %H:%M:%S')

display(df_clima)


,cidade,codigo_pais,temperatura,sensacao_termica,temperatura_min,temperatura_max,umidade,descricao_clima,horario_medicao
0,Tripoli,LY,22.92,22.47,22.92,22.92,46,nuvens dispersas,03/05/2025 16:17:46
1,Luxembourg,LU,18.84,18.66,17.72,19.62,72,nuvens dispersas,03/05/2025 16:18:11
2,Ankara,TR,16.23,15.21,15.51,16.23,50,nublado,03/05/2025 16:18:13
3,London,GB,16.13,15.34,14.78,17.98,59,nublado,03/05/2025 16:15:40
4,Saint-Pierre,PM,3.98,-0.01,3.98,3.98,100,névoa,03/05/2025 16:25:58
...,...,...,...,...,...,...,...,...,...
178,Dublin,IE,13.64,12.73,12.71,14.76,64,algumas nuvens,03/05/2025 17:58:08
179,Dakar,SN,25.07,25.54,25.07,25.07,73,chuva leve,03/05/2025 17:57:53
180,Honiara,SB,25.42,26.26,25.42,25.42,86,nublado,03/05/2025 18:00:19
181,Ramallah,PS,15.88,15.48,15.85,15.88,75,algumas nuvens,03/05/2025 18:00:21


In [13]:
from pandasql import sqldf

# Define uma função para executar consultas SQL
pysqldf = lambda q: sqldf(q, globals())

In [16]:
consultaSQL = """

SELECT * FROM df_clima 
--where codigo_pais = "AR"

"""

resultado = pysqldf(consultaSQL)
display(resultado)

,cidade,codigo_pais,temperatura,sensacao_termica,temperatura_min,temperatura_max,umidade,descricao_clima,horario_medicao
0,Tripoli,LY,22.92,22.47,22.92,22.92,46,nuvens dispersas,03/05/2025 16:17:46
1,Luxembourg,LU,18.84,18.66,17.72,19.62,72,nuvens dispersas,03/05/2025 16:18:11
2,Ankara,TR,16.23,15.21,15.51,16.23,50,nublado,03/05/2025 16:18:13
3,London,GB,16.13,15.34,14.78,17.98,59,nublado,03/05/2025 16:15:40
4,Saint-Pierre,PM,3.98,-0.01,3.98,3.98,100,névoa,03/05/2025 16:25:58
...,...,...,...,...,...,...,...,...,...
178,Dublin,IE,13.64,12.73,12.71,14.76,64,algumas nuvens,03/05/2025 17:58:08
179,Dakar,SN,25.07,25.54,25.07,25.07,73,chuva leve,03/05/2025 17:57:53
180,Honiara,SB,25.42,26.26,25.42,25.42,86,nublado,03/05/2025 18:00:19
181,Ramallah,PS,15.88,15.48,15.85,15.88,75,algumas nuvens,03/05/2025 18:00:21


In [15]:
df_export = pd.DataFrame(resultado)  # Dados dim pais
df_export.to_csv("df_dim_tempo_capital.csv", index=False)

In [17]:
resultado.dtypes

cidade               object
codigo_pais          object
temperatura         float64
sensacao_termica    float64
temperatura_min     float64
temperatura_max     float64
umidade               int64
descricao_clima      object
horario_medicao      object
dtype: object

In [ ]:
import psycopg2

# Parâmetros de conexão
dbname   = 'projeto_paises'
user     = 'postgres'
password = '252500'
host     = 'localhost'
port     = '5432' 

# Criar uma conexão
conn = psycopg2.connect(dbname=dbname,
                        user=user,
                        password=password,
                        host=host,
                        port=port)


cur = conn.cursor()# Criar um cursor  deixa manipular os dados
#cur.execute("DELETE FROM TBL_CLIMA_CAPITAL_PAIS")

for indice, colunas_df in resultado.iterrows():
        cur.execute('''                                
                insert into TBL_CLIMA_CAPITAL_PAIS (   
                 cidade,
                 codigo_pais,
                 temperatura,
                 sensacao_termica,
                 temperatura_min,
                 temperatura_max,
                 umidade,
                 descricao_clima,
                 horario_medicao)
                 VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
                        ''',
        (
        colunas_df["cidade"],
        colunas_df["codigo_pais"],
        colunas_df["temperatura"],
        colunas_df["sensacao_termica"],
        colunas_df["temperatura_min"],
        colunas_df["temperatura_max"],
        colunas_df["umidade"],
        colunas_df["descricao_clima"],
        colunas_df["horario_medicao"]
        )
        )

#comandos sql transct-sql
conn.commit() # validar alterações que agente fez  e subir para o banco de dados 

# Fechar o cursor e a conexão
cur.close()
conn.close()